In [1]:
from bs4 import BeautifulSoup, Tag
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys

import openpyxl
import time
import os
import re

from dotenv import load_dotenv #python-dotenv
load_dotenv()

True

In [2]:
mangalib_id = os.environ.get('MANGALIB_ID')

def create_bookmarks_dict_mangalib(soup, driver):
    """Возвращает словарь закладок"""
    full_bookmarks = soup.findAll('div', {'class': 'menu__item'})
    bookmarks_dict = {}
    
    for i in range(2,len(full_bookmarks)-1):
        driver.find_element('xpath',f'//*[@id="main-page"]/div/div/div[2]/div[1]/div/div[1]/div[1]/div[{i}]').click()
        id = driver.current_url.split('=')[1]
        data = full_bookmarks[i-1].text.strip().rstrip().lstrip()
        data = re.split(r'(\d+)', data.strip())
        data[0] = data[0].strip()
        data = data[0:2]
        
        data.append(int(data[1])//17 + 2)
        bookmarks_dict[id] = data
    
    del bookmarks_dict['2'] #В планах
    return bookmarks_dict


def check_bookmarks_dict_mangalib(bookmarks_dict, driver):
    """проверка bookmarks_dir, созданного create_bookmarks_dir_mangalib()"""
    for id, value in bookmarks_dict.items():

        #открываем сайт закладки
        driver.switch_to.new_window()
        driver.get(f'https://mangalib.me/user/{mangalib_id}?folder={id}')
        time.sleep(1)

        #переход к нижней части прогруженной страницы
        scrolls = 0
        while scrolls < value[2]:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2)
            scrolls += 1

        #получение всех div с классом '', содержат информацию по тайтлам
        soup_bookmarks = BeautifulSoup(driver.page_source)
        title_blocks = soup_bookmarks.findAll('div', {'class': 'bookmark-item'})

        #[url на тайтл, дата добавления в закладки]
        title_list = []
        for block in title_blocks:
            url = block.find('a')['href']
            added = block.find('div', {'class': 'bookmark-meta-item__value'}).text
            added = added.rstrip().lstrip()
            title_list.append([url, added])

        if len(title_blocks) != int(value[1]):
            print(f'Error: Закладка "{value[0]}"\n Ожидалось {value[1]}, получено {len(title_list)} тайтлов\n    Измените {value[2]} - количество scrolls в bookmarks_dict')
            break

        #закрываем сайт закладки, возвращаемся на исходную вкладку
        driver.close()
        driver.switch_to.window(driver.window_handles[0])

    print('Количество scroll подобрано правильно')


def delete_cache(driver):
    """очистка кэша driver"""
    driver.switch_to.new_window() # открываем новую вкладку 
    driver.get('chrome://settings/clearBrowserData') # Открываем настройки chrome
    time.sleep(0.2)
    actions = ActionChains(driver)
    actions.key_down(Keys.SHIFT).send_keys(Keys.TAB * 6).key_up(Keys.SHIFT)  #select "all time" browsing data
    actions.send_keys(Keys.DOWN * 5 +  Keys.TAB * 7 + Keys.ENTER)  #click on "clear data" button
    time.sleep(0.2)
    
    # закрываем новую вкладку, возвращаемся на исходную вкладку
    driver.close()  
    driver.switch_to.window(driver.window_handles[0])  
    
    
def parse_to_excel(bookmarks_dict, driver):
    """Запись информации по тайтлам в excel"""
    
    wb = openpyxl.Workbook()
    ws = wb.active
    ws.append(['Name_ru', 'Name_eng', 'Type', 'Date_added', 'Alternative_names'])
    
    num_cache = 1
    for id, value in bookmarks_dict.items():
        driver.get(f'https://mangalib.me/user/{mangalib_id}?folder={id}') #открываем сайт закладки
        time.sleep(0.2)
    
        #переход к нижней части страницы
        scrolls = 0
        while scrolls < value[2]:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(0.5)
            scrolls += 1
        time.sleep(0.1)
        
        #получение всех div с классом '', содержат информацию по тайтлам
        location_blocks = BeautifulSoup(driver.page_source).findAll('div', {'class': 'bookmark-item'})
        for block in location_blocks:
            url = block.find('a')['href'].split('=')[0]+'=info'
            added = block.find('div', {'class': 'bookmark-meta-item__value'}).text.rstrip().lstrip()
            driver.get(f'https://mangalib.me{url}')
            time.sleep(2)
            
            #данные
            title_data = BeautifulSoup(driver.page_source)
            name_ru = title_data.find('div', {'class': 'media-name__main'})
            name_eng = title_data.find('div', {'class': 'media-name__alt'})
            title_type = title_data.find('div', {'class': 'media-info-list__value'})
            alter_names_soup = title_data.find('div', {'class': 'media-info-list__item media-info-list__item_alt-names'})
            if isinstance(alter_names_soup, Tag):
                alter_names = alter_names_soup.contents[3].getText().rstrip().lstrip().replace("\n", ";")
            else: alter_names = ''
                
            result = [name_ru.text, name_eng.text, title_type.text, added, alter_names]
            ws.append(result)
            time.sleep(0.3)
            
            # очистка кэша
            if num_cache % 30==0:
                delete_cache(driver)
            num_cache+=1
            
    wb.save('parse-data/parse_mangalib1.xlsx')
    
    print('Данные сохранены')

In [3]:
email = os.environ.get('GOOGLE_LOG')
password = os.environ.get('GOOGLE_PASS')

#запуск Chrome
options = uc.ChromeOptions()
options.page_load_strategy = 'none'
chrome = uc.Chrome(options=options)

#авторизация Google
chrome.get('https://accounts.google.com/')
time.sleep(1)
chrome.find_element(By.XPATH, '//*[@id="identifierId"]').send_keys(email)
time.sleep(0.5)
chrome.find_element(By.XPATH, '//*[@id="identifierNext"]/div/button/span').click()
time.sleep(10)
chrome.find_element(By.XPATH, '//*[@id="password"]/div[1]/div/div[1]/input').send_keys(password)
time.sleep(0.5)
chrome.find_element(By.XPATH, '//*[@id="passwordNext"]/div/button/span').click()
#в моем случае надо зайти в Youtube и подтвердить 

In [4]:
#авторизация
chrome.get(f'https://lib.social/auth/google?from=http%3A%2F%2Fmangalib.me%2F%3Fsection%3Dhome-updates-{mangalib_id}')
time.sleep(5)
chrome.get(f'https://mangalib.me/user/{mangalib_id}?folder=all') #закладки "Все"

In [5]:
soup_ = BeautifulSoup(chrome.page_source)
time.sleep(0.5)

#parse
bookmarks_dict1 = create_bookmarks_dict_mangalib(soup_, chrome)
# check_bookmarks_dict_mangalib(bookmarks_dict1, driver)
parse_to_excel(bookmarks_dict1, chrome)

Данные сохранены


In [ ]:
#закрыть браузер
chrome.quit()